Establecemos una ``semilla`` que nos permitirá hacer que los experimentos sean reproducibles.

In [1]:
#semilla
seed = 42

Realizamos todas las `importaciones` necesarias.

In [12]:
#Importamos las librerias
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import get_scorer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn import set_config
import cloudpickle #para guardar y cargar modelos
import time
import matplotlib.pyplot as plt

import pandas as pd

import os
from PIL import Image

# 1.Carga de datos

In [13]:
image_dir = 'sin_procesar'

images = []

for filename in os.listdir(image_dir):
    if filename.endswith('.jpg'): 
        img_path = os.path.join(image_dir, filename)
        img = Image.open(img_path)
        images.append(img)

# Print the number of images loaded
print(f'Loaded {len(images)} images.')
#indica el nombre de la imagen en la posicion 2
print(images[2].filename)

Loaded 70 images.
sin_procesar\11.jpg


In [ ]:
set_config(transform_output="pandas")
df = pd.read_csv('hit2.csv')

Comprobando que se ha cargado correctamente:

In [116]:
df.sample(5, random_state=seed)

,time_left,ct_score,t_score,map,bomb_planted,ct_health,t_health,ct_armor,t_armor,ct_money,...,t_grenade_flashbang,ct_grenade_smokegrenade,t_grenade_smokegrenade,ct_grenade_incendiarygrenade,t_grenade_incendiarygrenade,ct_grenade_molotovgrenade,t_grenade_molotovgrenade,ct_grenade_decoygrenade,t_grenade_decoygrenade,round_winner
47053,175.00,11.0,4.0,de_vertigo,False,500.0,500.0,400.0,300.0,700.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,CT
28740,94.95,8.0,4.0,de_dust2,False,400.0,450.0,400.0,477.0,10300.0,...,2.0,2.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,CT
92746,14.93,14.0,10.0,de_dust2,False,366.0,100.0,395.0,98.0,2350.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,CT
60470,46.27,12.0,15.0,de_nuke,False,400.0,446.0,399.0,434.0,300.0,...,5.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,T
42953,37.07,18.0,18.0,de_train,False,100.0,200.0,100.0,200.0,10100.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,T


A continuación, creamos ``etiquetas`` para separar la variable objetivo del resto de variables.

In [ ]:
target = " "
columns = target
axis = "columns"

arguments = {"columns": columns, "axis": axis}
X, y = df.drop(**arguments), df[target]

Vamos a comprobar que se ha separado correctamente. Comenzamos con las `variables predictoras`:

In [118]:
X.sample(5, random_state=seed)

,time_left,ct_score,t_score,map,bomb_planted,ct_health,t_health,ct_armor,t_armor,ct_money,...,ct_grenade_flashbang,t_grenade_flashbang,ct_grenade_smokegrenade,t_grenade_smokegrenade,ct_grenade_incendiarygrenade,t_grenade_incendiarygrenade,ct_grenade_molotovgrenade,t_grenade_molotovgrenade,ct_grenade_decoygrenade,t_grenade_decoygrenade
47053,175.00,11.0,4.0,de_vertigo,False,500.0,500.0,400.0,300.0,700.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
28740,94.95,8.0,4.0,de_dust2,False,400.0,450.0,400.0,477.0,10300.0,...,4.0,2.0,2.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0
92746,14.93,14.0,10.0,de_dust2,False,366.0,100.0,395.0,98.0,2350.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
60470,46.27,12.0,15.0,de_nuke,False,400.0,446.0,399.0,434.0,300.0,...,1.0,5.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0
42953,37.07,18.0,18.0,de_train,False,100.0,200.0,100.0,200.0,10100.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


Y continuamos con la `variable clase`:

In [119]:
y.sample(5, random_state=seed)

47053    CT
28740    CT
92746    CT
60470     T
42953     T
Name: round_winner, dtype: object

Por último, `dividimos` el conjunto de datos en entrenamiento y prueba mediante un *holdout* estratificado:

In [120]:
test_size = 0.3

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed, stratify=y)

Y nos aseguramos que se ha realizado adecuadamente. Comenzamos con el conjunto de datos de entrenamiento:

In [121]:
X_train.sample(5, random_state=seed)

,time_left,ct_score,t_score,map,bomb_planted,ct_health,t_health,ct_armor,t_armor,ct_money,...,ct_grenade_flashbang,t_grenade_flashbang,ct_grenade_smokegrenade,t_grenade_smokegrenade,ct_grenade_incendiarygrenade,t_grenade_incendiarygrenade,ct_grenade_molotovgrenade,t_grenade_molotovgrenade,ct_grenade_decoygrenade,t_grenade_decoygrenade
4136,27.62,7.0,9.0,de_dust2,True,236.0,0.0,261.0,0.0,950.0,...,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
17231,94.80,8.0,5.0,de_vertigo,False,379.0,400.0,100.0,400.0,11400.0,...,0.0,1.0,0.0,3.0,0.0,0.0,0.0,2.0,0.0,0.0
84781,29.94,8.0,9.0,de_nuke,True,73.0,162.0,95.0,188.0,3300.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
18433,164.95,5.0,12.0,de_dust2,False,500.0,500.0,393.0,483.0,2750.0,...,1.0,5.0,2.0,5.0,0.0,0.0,0.0,5.0,0.0,0.0
17320,114.87,14.0,14.0,de_vertigo,False,500.0,500.0,500.0,410.0,350.0,...,3.0,5.0,4.0,5.0,0.0,0.0,0.0,5.0,2.0,0.0


In [122]:
y_train.sample(5, random_state=seed)

4136     CT
17231     T
84781     T
18433    CT
17320     T
Name: round_winner, dtype: object

Y finalizamos con el conjunto de datos de prueba:

In [123]:
X_test.sample(5, random_state=seed)

,time_left,ct_score,t_score,map,bomb_planted,ct_health,t_health,ct_armor,t_armor,ct_money,...,ct_grenade_flashbang,t_grenade_flashbang,ct_grenade_smokegrenade,t_grenade_smokegrenade,ct_grenade_incendiarygrenade,t_grenade_incendiarygrenade,ct_grenade_molotovgrenade,t_grenade_molotovgrenade,ct_grenade_decoygrenade,t_grenade_decoygrenade
55536,74.90,3.0,5.0,de_overpass,False,300.0,304.0,200.0,343.0,200.0,...,0.0,3.0,1.0,4.0,1.0,0.0,0.0,3.0,0.0,0.0
86184,94.92,1.0,8.0,de_overpass,False,300.0,412.0,300.0,392.0,550.0,...,2.0,5.0,2.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0
14258,169.94,3.0,10.0,de_overpass,False,500.0,500.0,0.0,93.0,18050.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
120453,34.51,10.0,7.0,de_dust2,True,229.0,0.0,295.0,0.0,8100.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
95891,174.91,1.0,5.0,de_inferno,False,500.0,500.0,400.0,0.0,17600.0,...,3.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [124]:
y_test.sample(5, random_state=seed)

55536     CT
86184      T
14258      T
120453    CT
95891     CT
Name: round_winner, dtype: object

Se borran las etiquetas que no aportan informacion

In [ ]:
etiquetas=[' ']

In [126]:
print(etiquetas)

['ct_score', 't_score', 'bomb_planted', 'ct_health', 't_health', 'ct_armor', 't_armor', 'ct_money', 't_money', 'ct_helmets', 't_helmets', 'ct_defuse_kits', 'ct_players_alive', 't_players_alive', 'ct_weapon_ak47', 't_weapon_ak47', 'ct_weapon_awp', 't_weapon_awp', 't_weapon_glock', 'ct_weapon_m4a4', 'ct_weapon_sg553', 't_weapon_sg553', 'ct_weapon_deagle', 't_weapon_deagle', 'ct_weapon_usps', 't_weapon_p250', 'ct_weapon_p2000', 'map']


# ELIMINACIÓN DE VARIABLES

vamos a `eliminar` las variables que no aportan información con la ayuda de un `pipeline`.

In [16]:
columns = etiquetas
condition = ~X.columns.isin(columns)
columns = X.columns[condition]
drop = make_column_transformer(("drop", columns), remainder="passthrough", verbose_feature_names_out=False)
drop

NameError: name 'etiquetas' is not defined

# CREACION DEL ENCODER

Tratamos las variables categóricas con un `encoder` para poder trabajar con ellas.

In [14]:
encoder = make_column_transformer(
    (OrdinalEncoder(categories=[["No", "Si"]]), ["Inuncada,Transitable"]),  # Convierte "No" -> 0, "Si" -> 1
    remainder="passthrough",
    verbose_feature_names_out=False
)
encoder

ColumnTransformer(remainder='passthrough',
                  transformers=[('ordinalencoder',
                                 OrdinalEncoder(categories=[['No', 'Si']]),
                                 ['Inuncada,Transitable'])],
                  verbose_feature_names_out=False)

# CREACION DE LAS TUPLAS POR ROTADO

# CREACION DEL PIPELINE

In [45]:
preprocessing = make_pipeline(encoder, drop)
preprocessing

NameError: name 'drop' is not defined

# TRANSFORMACION DE LAS IMÁGENES

In [ ]:
# Obtener los nombres de las imágenes en X_train
##image_names = X_train['image_name_column'].tolist()  # Reemplaza 'image_name_column' con el nombre real de la columna que contiene los nombres de las imágenes
image_names="sin_procesar\\11.jpg"
# Imprimir los nombres de las imágenes
print(image_names)
# recorremos la lista de nombres de las images y creamos una lista con las imagenes de image_names
images_train = []
for img in images:
    if img.filename in image_names:
        img.filename = img.filename.split("\\")[-1]
        images_train.append(img)
images_train[0].filename

sin_procesar\11.jpg


'11.jpg'

In [ ]:
# Convertir imágenes a escala de grises y rotarlas
augmented_images = []

for img in images_train:
    # Convertir a escala de grises
    gray_img = img.convert('L')
    gray_img.filename = "0_"+img.filename
    
    # Rotar la imagen 90º, 180º y 270º
    rotated_90 = gray_img.rotate(90)
    rotated_180 = gray_img.rotate(180)
    rotated_270 = gray_img.rotate(270)
    rotated_90.filename = "90_"+img.filename
    rotated_180.filename = "180_"+img.filename
    rotated_270.filename = "270_"+img.filename
    
    # Añadir las imágenes originales y transformadas a la lista con filenames
    augmented_images.extend([gray_img, rotated_90, rotated_180, rotated_270])

# Imprimir el número de imágenes aumentadas
print(f'Número total de imágenes aumentadas: {len(augmented_images)}')
augmented_images[2].filename

Número total de imágenes aumentadas: 4


'180_11.jpg'

In [47]:
output_dir = 'trainprocesados'
os.makedirs(output_dir, exist_ok=True)

for img in augmented_images:
    img.save(os.path.join(output_dir, img.filename))

print(f'Se han guardado {len(augmented_images)} imágenes en la carpeta {output_dir}.')

Se han guardado 4 imágenes en la carpeta trainprocesados.
